<a href="https://colab.research.google.com/github/IO-2024/IOTs_with_python/blob/main/Ch7.%20%EC%A7%80%EC%97%AD%20%EA%B0%84%20%EC%82%B0%EC%97%85%EC%97%B0%EA%B4%80%ED%9A%A8%EA%B3%BC%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

In [2]:
np.set_printoptions(precision=3,suppress=True) # 결과를 소수 3자리까지만 보여주는 옵션

In [3]:
Rgt = "https://raw.githubusercontent.com/IO-2024/IOTs_with_python/main/data/Regional_IO_3regions_3sectors.csv"  # 지역 간 산업연관표(3지역, 3부문) 불러오기
Rgt_3 = pd.read_csv(Rgt)
Rgt_3.head()

,rg,com,수도권,수도권.1,수도권.2,동부권,동부권.1,동부권.2,서부권,서부권.1,...,동부권.3,동부권.4,동부권.5,서부권.3,서부권.4,서부권.5,지역최종수요계,지역총수요계,수입,총산출액
0,rg,com,농산품,공산품,서비스,농산품,공산품,서비스,농산품,공산품,...,소비,투자,수출,소비,투자,수출,제품최종수요계,제품총수요계,(공제),총산출액
1,수도권,농산품,0,3,1,0,0,0,0,1,...,0,0,0,0,0,0,1,6,0,6
2,수도권,공산품,2,112,71,1,36,14,1,37,...,9,6,0,7,8,0,295,582,0,582
3,수도권,서비스,1,79,302,1,27,31,1,28,...,33,13,0,31,11,0,706,1204,0,1204
4,동부권,농산품,0,1,2,1,7,2,0,2,...,3,1,0,1,0,0,8,23,0,23


In [4]:
Rgt_empt = "https://raw.githubusercontent.com/IO-2024/IOTs_with_python/main/data/Regional_IO_3sector_3rg_emp.csv"  # 지역별 고용표(3지역, 3부문) 불러오기
Rgt_emp_3 = pd.read_csv(Rgt_empt)
Rgt_emp_3.head()

,cd 수도권,동부권,서부권
농산품,135543.0,485642.0,606337.0
공산품,1818636.0,1444100.0,834780.0
서비스,10302991.0,4744657.0,3749541.0


## 지역, 산업구조 분석

In [5]:
Rgt_33 = Rgt_3.to_numpy()
Mt = Rgt_33[1:,2:].astype(float)
Mt

array([[   0.,    3.,    1.,    0.,    0.,    0.,    0.,    1.,    0.,
           5.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    1.,    6.,    0.,    6.],
       [   2.,  112.,   71.,    1.,   36.,   14.,    1.,   37.,   13.,
         287.,   29.,   21.,  215.,    9.,    6.,    0.,    7.,    8.,
           0.,  295.,  582.,    0.,  582.],
       [   1.,   79.,  302.,    1.,   27.,   31.,    1.,   28.,   28.,
         498.,  387.,  164.,   67.,   33.,   13.,    0.,   31.,   11.,
           0.,  706., 1204.,    0., 1204.],
       [   0.,    1.,    2.,    1.,    7.,    2.,    0.,    2.,    0.,
          15.,    3.,    0.,    0.,    3.,    1.,    0.,    1.,    0.,
           0.,    8.,   23.,    0.,   23.],
       [   0.,   43.,   38.,    4.,  158.,   40.,    1.,   36.,   13.,
         333.,   20.,   15.,    0.,   19.,   11.,  226.,    7.,    7.,
           0.,  305.,  638.,    0.,  638.],
       [   0.,    4.,   15.,    2.,   59.,   76.,    0.,    3.,    4.,

In [6]:
Va = Mt[-2,:10].reshape([1,10]) # 지역별, 부문별 부가가치 벡터
Ct = Mt[-1,:10].reshape([1,10])  # 지역별, 부문별 총산출액 벡터
Ct

array([[   6.,  582., 1204.,   23.,  638.,  479.,   29.,  491.,  379.,
        3831.]])

In [7]:
# 지역기준
Sudo = np.vstack((Va[:,:3].sum(),Ct[:,:3].sum()))      # 수도권
Dongbu = np.vstack((Va[:,3:6].sum(),Ct[:,3:6].sum()))  # 동부권
Seobu = np.vstack((Va[:,6:9].sum(),Ct[:,6:9].sum()))   # 서부권

Sm_vact = np.vstack((Va[:,9],Ct[:,9]))
Vact_all = np.hstack((Sudo, Dongbu, Seobu, Sm_vact))

Va_sm = np.hstack((Sudo, Dongbu, Seobu))
Ctva_wgt = (Vact_all/Sm_vact)
Str_rg = np.vstack((Vact_all,Ctva_wgt))
Str_rg.round(1)

array([[ 831. ,  447. ,  359. , 1637. ],
       [1792. , 1140. ,  899. , 3831. ],
       [   0.5,    0.3,    0.2,    1. ],
       [   0.5,    0.3,    0.2,    1. ]])

### 지역별 산업 비중

In [8]:
# 지역별 산업비중(총산출액 기준)
Sudo_ind_by_ct = (Mt[-1,:3]/Str_rg[1,0]*100) # 수도권
Dongbu_ind_by_ct = (Mt[-1,3:6]/Str_rg[1,1]*100) # 동부권

Seobu_ind_by_ct = (Mt[-1,6:9]/Str_rg[1,2]*100) # 동부권
Tot_ind_by_ct =  ((Mt[-1,:3]+Mt[-1,3:6]+Mt[-1,6:9])/Vact_all[1,3]*100)  # 전지역

Wgt_ind_by_region_ct = np.vstack((Sudo_ind_by_ct,Dongbu_ind_by_ct,Seobu_ind_by_ct, Tot_ind_by_ct))
print('지역별 산업 비중(총산출액 기준)  \n',Wgt_ind_by_region_ct.round(1))

지역별 산업 비중(총산출액 기준)  
 [[ 0.3 32.5 67.2]
 [ 2.  56.  42. ]
 [ 3.2 54.6 42.2]
 [ 1.5 44.7 53.8]]


In [9]:
# 지역별 산업비중(부가가치 기준)

Sudo_ind_by_va = (Mt[-2,:3]/Str_rg[0,0]*100) # 수도권
Dongbu_ind_by_va = (Mt[-2,3:6]/Str_rg[0,1]*100) # 동부권
Seobu_ind_by_va = (Mt[-2,6:9]/Str_rg[0,2]*100) # 서부권
Tot_ind_by_va =  ((Mt[-2,:3]+Mt[-2,3:6]+Mt[-2,6:9])/Vact_all[0,3]*100)  # 전지역

Wgt_ind_by_region_va = np.vstack((Sudo_ind_by_va,Dongbu_ind_by_va,Seobu_ind_by_va,Tot_ind_by_va))
print('지역별 산업 비중(부가가치 기준)  \n',Wgt_ind_by_region_va.round(1))

지역별 산업 비중(부가가치 기준)  
 [[ 0.4 21.5 78.1]
 [ 2.9 37.6 59.5]
 [ 4.5 38.2 57.4]
 [ 2.  29.6 68.5]]


In [10]:
CT_Reg = pd.DataFrame(Wgt_ind_by_region_ct.round(1), columns = ['농산품', '공산품', '서비스'],
                     index=['수도권', '동부권', '서부권','전지역']) # 변수명 붙여서 pandas로 내보내기
CT_Reg

,농산품,공산품,서비스
수도권,0.3,32.5,67.2
동부권,2.0,56.0,42.0
서부권,3.2,54.6,42.2
전지역,1.5,44.7,53.8


In [11]:
VA_Reg = pd.DataFrame(Wgt_ind_by_region_va.round(1), columns = ['수도권', '동부권', '서부권'],
                     index=['수도권', '동부권', '서부권','전지역']) # 변수명 붙여서 pandas로 내보내기
VA_Reg

,수도권,동부권,서부권
수도권,0.4,21.5,78.1
동부권,2.9,37.6,59.5
서부권,4.5,38.2,57.4
전지역,2.0,29.6,68.5


### 지역별 산업 입지계수

In [12]:
X_ij = Mt[-1,:9].reshape([1,9])
X = X_ij.sum()

In [13]:
X_sudo =X_ij[-1,:3].sum().reshape([1,1]) # 수도권 총산출액
X_seobu =X_ij[-1,3:6].sum().reshape([1,1]) # 동부권 총산출액
X_dongbu =X_ij[-1,6:9].sum().reshape([1,1]) # 서부권 총산출액
print(X_sudo,X_seobu,X_dongbu)

[[1792.]] [[1140.]] [[899.]]


In [14]:
X_agr=X_ij[-1,0]+X_ij[-1,3]+X_ij[-1,6] # 전국 농산품 총산출액
X_manu=X_ij[-1,1]+X_ij[-1,4]+X_ij[-1,7] # 전국 공산품 총산출액
X_serv=X_ij[-1,2]+X_ij[-1,5]+X_ij[-1,8] # 전국 서비스 총산출액
print(X_agr,X_manu, X_serv)

58.0 1711.0 2062.0


In [15]:
X_i = np.concatenate((np.tile(X_sudo,3),np.tile(X_seobu,3),np.tile(X_dongbu,3)),axis=1)  # 지역별 산출액 합

In [16]:
X_j = np.concatenate((X_agr.reshape([1,1]),X_manu.reshape([1,1]),X_serv.reshape([1,1])),axis=1) # 전국 농산품, 공산품, 서비스 산출액
X_j

array([[  58., 1711., 2062.]])

In [17]:
X_ij_div_by_Xi=X_ij/X_i
X_ij_div_by_Xi

array([[0.003, 0.325, 0.672, 0.02 , 0.56 , 0.42 , 0.032, 0.546, 0.422]])

In [18]:
Xj_div_by_X = np.tile(X_j/X,3).reshape(1,9)
Xj_div_by_X

array([[0.015, 0.447, 0.538, 0.015, 0.447, 0.538, 0.015, 0.447, 0.538]])

In [19]:
LQ = (X_ij_div_by_Xi/Xj_div_by_X).reshape([1,9]).round(3)
LQ_f = np.vstack((LQ[:,:3],LQ[:,3:6],LQ[:,6:9])) #  지역별 산업별 비중
print('지역별 산업 입지계수 \n', LQ_f)

지역별 산업 입지계수 
 [[0.221 0.727 1.248]
 [1.333 1.253 0.781]
 [2.131 1.223 0.783]]


### 산업별 지역 비중 구하기

In [20]:
# 산업기준

# 지역별 산출액 벡터
Arg_ct = np.hstack((Ct[:,0],Ct[:,3],Ct[:,6]))    # 농산품
Manu_ct = np.hstack((Ct[:,1],Ct[:,4],Ct[:,7]))  # 공산품
Serv_ct = np.hstack((Ct[:,2],Ct[:,5],Ct[:,8]))  # 서비스

# 지역별 부가가치 벡터
Arg_va = np.hstack((Va[:,0],Va[:,3],Va[:,6]))    # 농산품
Manu_va = np.hstack((Va[:,1],Va[:,4],Va[:,7]))  # 공산품
Serv_va= np.hstack((Va[:,2],Va[:,5],Va[:,8]))  # 서비스

In [21]:
Arg_ct_sm = np.sum(Arg_ct)
Manu_ct_sm = np.sum(Manu_ct)
Serv_ct_sm = np.sum(Serv_ct)

Arg_va_sm = np.sum(Arg_va)
Manu_va_sm = np.sum(Manu_va)
Serv_va_sm = np.sum(Serv_va)

In [22]:
# 산업별 지역 비중(총산출액 기준)
Arg_ct_wgt = (Arg_ct/Arg_ct_sm)  # 농산품
Manu_ct_wgt = (Manu_ct/Manu_ct_sm)  # 공산품
Serv_ct_wgt = (Serv_ct/Serv_ct_sm)  # 서비스
Serv_ct_wgt

array([0.584, 0.232, 0.184])

In [23]:
# 산업별 지역 비중(부가가치 기준)
Arg_va_wgt = (Arg_va/Arg_va_sm)  # 농산품
Manu_va_wgt = (Manu_va/Manu_va_sm)  # 공산품
Serv_va_wgt = (Serv_va/Serv_va_sm)  # 서비스
Serv_va_wgt

array([0.579, 0.237, 0.184])

In [24]:
Ct_by_ind = np.vstack((Arg_ct_wgt, Manu_ct_wgt, Serv_ct_wgt))
Va_by_ind = np.vstack((Arg_va_wgt, Manu_va_wgt, Serv_va_wgt))
print('산업별 지역 비중(총산출액 기준) \n',(Ct_by_ind*100).round(1), '\n')
print('산업별 지역 비중(부가가치 기준) \n', (Va_by_ind*100).round(1))

산업별 지역 비중(총산출액 기준) 
 [[10.3 39.7 50. ]
 [34.  37.3 28.7]
 [58.4 23.2 18.4]] 

산업별 지역 비중(부가가치 기준) 
 [[ 9.4 40.6 50. ]
 [37.  34.7 28.3]
 [57.9 23.7 18.4]]


In [25]:
CT_IND = pd.DataFrame((Ct_by_ind*100).round(1), columns = ['수도권', '동부권', '서부권'],
                     index=['농산품', '공산품', '서비스']) # 변수명 붙여서 pandas로 내보내기
CT_IND

,수도권,동부권,서부권
농산품,10.3,39.7,50.0
공산품,34.0,37.3,28.7
서비스,58.4,23.2,18.4


In [26]:
VA_IND = pd.DataFrame((Va_by_ind*100).round(1), columns = ['수도권', '동부권', '서부권'],
                     index=['농산품', '공산품', '서비스']) # 변수명 붙여서 pandas로 내보내기
VA_IND

,수도권,동부권,서부권
농산품,9.4,40.6,50.0
공산품,37.0,34.7,28.3
서비스,57.9,23.7,18.4


In [27]:
# csv 파일로 내보내기
VA_IND.to_csv('부가가치_산업별_지역별_비중.csv', encoding='utf_8-sig')
CT_IND.to_csv('산출액_산업별_지역별_비중.csv', encoding='utf_8-sig')

### 산업별 집중계수

In [28]:
Wgt_rg = Str_rg[-1,:3].reshape([1,3])   # str_rg는 지역별 총산출액 및 총부가가치의 금액과 비중(수도권, 동부권, 서부권)
Wgt_rg

array([[0.468, 0.298, 0.235]])

In [29]:
Arg_concst = (abs(Wgt_rg[:,0]-Ct_by_ind[0,0])+abs(Wgt_rg[:,1]-Ct_by_ind[0,1])+abs(Wgt_rg[:,2]-Ct_by_ind[0,2]))/2  # 농산품
Manu_concst = (abs(Wgt_rg[:,0]-Ct_by_ind[1,0])+abs(Wgt_rg[:,1]-Ct_by_ind[1,1])+abs(Wgt_rg[:,2]-Ct_by_ind[1,2]))/2  # 공산품
Serv_concst = (abs(Wgt_rg[:,0]-Ct_by_ind[2,0])+abs(Wgt_rg[:,1]-Ct_by_ind[2,1])+abs(Wgt_rg[:,2]-Ct_by_ind[2,2]))/2  # 서비스

In [30]:
All_concst = np.vstack((Arg_concst, Manu_concst, Serv_concst))  # 집중계수
print('산업별 집중계수 \n', All_concst.round(3))

산업별 집중계수 
 [[0.364]
 [0.128]
 [0.116]]


In [31]:
All_concst_df = pd.DataFrame(All_concst.round(3), columns = ['집중계수'],
                     index=['농산품', '공산품', '서비스']) # 변수명 붙여서 pandas로 내보내기
All_concst_df

,집중계수
농산품,0.364
공산품,0.128
서비스,0.116


### 지역별 투입구조

In [32]:
Dm_1 = Mt[0:9,0:9].astype(float)
Im_1 = Mt[9:12,0:9].astype(float)

In [33]:
import scipy.linalg as linalg

In [34]:
Mtx_for_int= linalg.block_diag(np.ones((3,3)),np.ones((3,3)),np.ones((3,3)))
Mtx_for_inter=np.where(Mtx_for_int == 1, 0, 1)

In [35]:
Intra_input = Mtx_for_int*Dm_1  # 지역별 자기지역 생산품 투입액

In [36]:
Inter_input = Mtx_for_inter*Dm_1  # 지역별 타지역 생산품 투입액

In [37]:
Intra_rg_sm = Intra_input.sum(axis=0) # 지역별 부문별 자기지역 생산품 투입액계
Inter_rg_sm = Inter_input.sum(axis=0) # 지역별 부문별 타지역 생산품 투입액계
Imps_input =Im_1.sum(axis=0)  # 지역별 부문별 수입중간투입계
print(Intra_rg_sm)

[  3. 194. 374.   7. 224. 118.   9. 141.  89.]


In [38]:
print(Intra_rg_sm, '\n')
print(Inter_rg_sm, '\n')
print(Imps_input)

[  3. 194. 374.   7. 224. 118.   9. 141.  89.] 

[  0.  99. 115.   3. 108.  64.   3. 107.  58.] 

[  0. 110.  66.   0. 138.  31.   1. 106.  26.]


In [39]:
Code = np.array([[1,1,1,0,0,0,0,0,0],  # 행렬 곱을 이용한 부문 통합을 위해 0과 1로 구성된 임시의 3x33 매트릭스
       [0,0,0,1,1,1,0,0,0],
       [0,0,0,0,0,0,1,1,1]])
Code.T

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [40]:
Intra_tot = (Intra_rg_sm@Code.T).T  # 지역별 자기지역 생산품 투입액계
Inter_tot = (Inter_rg_sm@Code.T).T  # 지역별 타지역 생산품 투입액계
Imps_tot =  (Imps_input@Code.T).T  # 지역별 수입중간투입계
Intra_tot

array([571., 349., 239.])

In [41]:
Input_dom = Intra_tot + Inter_tot
Input_all = Input_dom + Imps_tot

Input_tot = np.vstack((Intra_tot, Inter_tot, Input_dom,Imps_tot, Input_all)).T
# input_tot_f = np.hstack((input_tot,input_tot.sum(axis=0).reshape([1,5])))
Input_tot.ndim

2

In [42]:
Input_nation = Input_tot.sum(axis=0).reshape([1,5])
Input_nation.ndim

2

In [43]:
Input_tem = np.vstack((Input_tot, Input_nation))
Input_fin = np.hstack((Input_tem,Str_rg[0,:].reshape([4,1])))
Input_fin

array([[ 571.,  214.,  785.,  176.,  961.,  831.],
       [ 349.,  175.,  524.,  169.,  693.,  447.],
       [ 239.,  168.,  407.,  133.,  540.,  359.],
       [1159.,  557., 1716.,  478., 2194., 1637.]])

In [44]:
Ct_rg = Str_rg[1,:]
Ct_rg_m = np.tile(Ct_rg, reps=[6,1]).T
Ct_rg_m

array([[1792., 1792., 1792., 1792., 1792., 1792.],
       [1140., 1140., 1140., 1140., 1140., 1140.],
       [ 899.,  899.,  899.,  899.,  899.,  899.],
       [3831., 3831., 3831., 3831., 3831., 3831.]])

In [45]:
Input_tot_wgt = Input_fin/Ct_rg_m
print('지역별 투입구조 \n', (Input_tot_wgt*100).round(1))

지역별 투입구조 
 [[31.9 11.9 43.8  9.8 53.6 46.4]
 [30.6 15.4 46.  14.8 60.8 39.2]
 [26.6 18.7 45.3 14.8 60.1 39.9]
 [30.3 14.5 44.8 12.5 57.3 42.7]]


In [46]:
Input_df = pd.DataFrame((Input_tot_wgt*100).round(1), columns = ['자기지역생산품','타지역생산품','국산품','수입품','중간투입계','부가가치'],
                     index=['수도권', '동부권', '서부권', '전지역']) # 변수명 붙여서 pandas로 내보내기
Input_df

,자기지역생산품,타지역생산품,국산품,수입품,중간투입계,부가가치
수도권,31.9,11.9,43.8,9.8,53.6,46.4
동부권,30.6,15.4,46.0,14.8,60.8,39.2
서부권,26.6,18.7,45.3,14.8,60.1,39.9
전지역,30.3,14.5,44.8,12.5,57.3,42.7


In [47]:
Input_df.to_csv('지역별 투입구조.csv', encoding='utf_8-sig')

## 지역별 배분구조

In [48]:
Intra_tot

array([571., 349., 239.])

In [49]:
Intra_itd_dstr = Intra_input.sum(axis=1)   # 지역별 부문별 자기지역 생산품 중간수요계
Inter_itd_dstr = Inter_input.sum(axis=1)   # 지역별 부문별 타지역 생산품 중간수요계
Itd_dstr_sm = Intra_itd_dstr + Inter_itd_dstr
Itd_dstr_sm

array([  5., 287., 498.,  15., 333., 163.,  22., 268., 125.])

In [50]:
Intra_itd_tot = (Intra_itd_dstr@Code.T)  # 지역별 부문별 자기지역 최종수요계
Inter_itd_tot = (Inter_itd_dstr@Code.T)  # 지역별 부문별 타지역 최종수요계
Itd_dstr_tot = (Itd_dstr_sm@Code.T)  # 지역별 중간수요계
Itd_dstr_tot

array([790., 511., 415.])

In [51]:
Int_dstr = np.vstack((Intra_itd_tot, Inter_itd_tot, Itd_dstr_tot)).T # 지역별 자기지역 중간수요, 타지역 중간수요, 중간수요계

In [52]:
Exp_sudo = Mt[:9,12].sum() # 지역별 총수출액(수도권)
Exp_dongbu = Mt[:9,15].sum() # 지역별 총수출액(동부권)
Exp_seobu = Mt[:9,18].sum() # 지역별 총수출액(서부권)

In [53]:
Exp_tot = np.hstack((Exp_sudo, Exp_dongbu, Exp_seobu))

In [54]:
print('지역별 자기지역 중간수요, 타지역 중간수요, 중간수요계','\n', Int_dstr, '\n')
print('지역별 총수출액', '\n', Exp_tot)

지역별 자기지역 중간수요, 타지역 중간수요, 중간수요계 
 [[571. 219. 790.]
 [349. 162. 511.]
 [239. 176. 415.]] 

지역별 총수출액 
 [282. 252. 180.]


In [55]:
Mt_for_nonex = np.copy(Mt)
Fd_rg = Mt_for_nonex[:9,10:19] # 지역별 부문별 최종수요 벡터
#수출을 0으로 만든 최종수요매트릭스 만들기
Fd_rg[:,2] = 0  # 수도권
Fd_rg[:,5] = 0  # 동부권
Fd_rg[:,8] = 0  # 서부권
Fd_rg

array([[  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 29.,  21.,   0.,   9.,   6.,   0.,   7.,   8.,   0.],
       [387., 164.,   0.,  33.,  13.,   0.,  31.,  11.,   0.],
       [  3.,   0.,   0.,   3.,   1.,   0.,   1.,   0.,   0.],
       [ 20.,  15.,   0.,  19.,  11.,   0.,   7.,   7.,   0.],
       [ 11.,   2.,   0., 194.,  77.,   0.,   5.,   1.,   0.],
       [  3.,   0.,   0.,   1.,   0.,   0.,   2.,   1.,   0.],
       [ 15.,   9.,   0.,   7.,   4.,   0.,  15.,   5.,   0.],
       [ 15.,   4.,   0.,   5.,   1.,   0., 146.,  71.,   0.]])

In [56]:
Intra_fd = (Mtx_for_int*Fd_rg).sum(axis=1) # 지역별 부문별 자기지역 국내 최종수요계
Intra_fd

array([  1.,  50., 551.,   4.,  30., 271.,   3.,  20., 217.])

In [57]:
Inter_fd =  (Mtx_for_inter*Fd_rg).sum(axis=1) # 지역별 부문별 타지역 국내 최종수요계

In [58]:
Tot_fd = Intra_fd + Inter_fd # 최종수요계

In [59]:
Intra_fd_tot = (Intra_fd@Code.T)  # 자기지역 최종종요 비중(전체산업)
Inter_fd_tot = (Inter_fd@Code.T)  # 타지역 최종종요 비중(전체산업)
Tot_fd_tot = (Tot_fd@Code.T)

In [60]:
Fd_dstr = np.vstack((Intra_fd_tot, Inter_fd_tot, Tot_fd_tot)).T

In [61]:
Total_dstr = np.hstack((Int_dstr, Fd_dstr, Exp_tot.reshape([3,1])))
Total_dstr

array([[571., 219., 790., 602., 118., 720., 282.],
       [349., 162., 511., 305.,  72., 377., 252.],
       [239., 176., 415., 240.,  64., 304., 180.]])

In [62]:
Tot_dstr_ct = Total_dstr[:,2]+Total_dstr[:,5]+Total_dstr[:,6]
Tot_dstr_ct

array([1792., 1140.,  899.])

In [63]:
Tot_dstr_col_sm = np.hstack((Total_dstr,Tot_dstr_ct.reshape([3,1])))
Tot_dstr_col_sm

array([[ 571.,  219.,  790.,  602.,  118.,  720.,  282., 1792.],
       [ 349.,  162.,  511.,  305.,   72.,  377.,  252., 1140.],
       [ 239.,  176.,  415.,  240.,   64.,  304.,  180.,  899.]])

In [64]:
Tot_dstr_rw_sm = Tot_dstr_col_sm.sum(axis=0)
Tot_dstr_rw_sm

array([1159.,  557., 1716., 1147.,  254., 1401.,  714., 3831.])

In [65]:
Tot_dstr = np.vstack((Tot_dstr_col_sm, Tot_dstr_rw_sm))
Tot_dstr

array([[ 571.,  219.,  790.,  602.,  118.,  720.,  282., 1792.],
       [ 349.,  162.,  511.,  305.,   72.,  377.,  252., 1140.],
       [ 239.,  176.,  415.,  240.,   64.,  304.,  180.,  899.],
       [1159.,  557., 1716., 1147.,  254., 1401.,  714., 3831.]])

In [66]:
Sm_col = np.tile(Tot_dstr[:,7].reshape([4,1]), reps=[1,8])
Sm_col

array([[1792., 1792., 1792., 1792., 1792., 1792., 1792., 1792.],
       [1140., 1140., 1140., 1140., 1140., 1140., 1140., 1140.],
       [ 899.,  899.,  899.,  899.,  899.,  899.,  899.,  899.],
       [3831., 3831., 3831., 3831., 3831., 3831., 3831., 3831.]])

In [67]:
Tot_dstr_ratio = Tot_dstr/Sm_col
print('지역별 배분구조', '\n', (Tot_dstr_ratio*100).round(1))

지역별 배분구조 
 [[ 31.9  12.2  44.1  33.6   6.6  40.2  15.7 100. ]
 [ 30.6  14.2  44.8  26.8   6.3  33.1  22.1 100. ]
 [ 26.6  19.6  46.2  26.7   7.1  33.8  20.  100. ]
 [ 30.3  14.5  44.8  29.9   6.6  36.6  18.6 100. ]]


In [68]:
dstr_df = pd.DataFrame((Tot_dstr_ratio*100).round(1), columns = [['중간재', '중간재', '중간재', '최종수요', '최종수요', '최종수요','수출', '합계'],
                                     ['자기지역', '타지역', '소계', '자기지역', '타지역', '소계','수출', '합계' ]],
                       index = ['수도권', '동부권', '서부권', '전지역'])
dstr_df

중간재              최종수요               수출     합계
     자기지역   타지역    소계  자기지역  타지역    소계    수출     합계
수도권  31.9  12.2  44.1  33.6  6.6  40.2  15.7  100.0
동부권  30.6  14.2  44.8  26.8  6.3  33.1  22.1  100.0
서부권  26.6  19.6  46.2  26.7  7.1  33.8  20.0  100.0
전지역  30.3  14.5  44.8  29.9  6.6  36.6  18.6  100.0

In [69]:
dstr_df.to_csv('지역별 배분구조.csv', encoding='utf_8-sig')

In [70]:
Mt

array([[   0.,    3.,    1.,    0.,    0.,    0.,    0.,    1.,    0.,
           5.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    1.,    6.,    0.,    6.],
       [   2.,  112.,   71.,    1.,   36.,   14.,    1.,   37.,   13.,
         287.,   29.,   21.,  215.,    9.,    6.,    0.,    7.,    8.,
           0.,  295.,  582.,    0.,  582.],
       [   1.,   79.,  302.,    1.,   27.,   31.,    1.,   28.,   28.,
         498.,  387.,  164.,   67.,   33.,   13.,    0.,   31.,   11.,
           0.,  706., 1204.,    0., 1204.],
       [   0.,    1.,    2.,    1.,    7.,    2.,    0.,    2.,    0.,
          15.,    3.,    0.,    0.,    3.,    1.,    0.,    1.,    0.,
           0.,    8.,   23.,    0.,   23.],
       [   0.,   43.,   38.,    4.,  158.,   40.,    1.,   36.,   13.,
         333.,   20.,   15.,    0.,   19.,   11.,  226.,    7.,    7.,
           0.,  305.,  638.,    0.,  638.],
       [   0.,    4.,   15.,    2.,   59.,   76.,    0.,    3.,    4.,

### 지역 이출입

In [71]:
Tra_int = Mt[0:9,0:9] # 지역별 부문별 중간투입(수요)
Tra_int

array([[  0.,   3.,   1.,   0.,   0.,   0.,   0.,   1.,   0.],
       [  2., 112.,  71.,   1.,  36.,  14.,   1.,  37.,  13.],
       [  1.,  79., 302.,   1.,  27.,  31.,   1.,  28.,  28.],
       [  0.,   1.,   2.,   1.,   7.,   2.,   0.,   2.,   0.],
       [  0.,  43.,  38.,   4., 158.,  40.,   1.,  36.,  13.],
       [  0.,   4.,  15.,   2.,  59.,  76.,   0.,   3.,   4.],
       [  0.,   2.,   3.,   0.,   1.,   1.,   1.,  12.,   2.],
       [  0.,  46.,  40.,   1.,  41.,  15.,   6.,  88.,  31.],
       [  0.,   3.,  17.,   0.,   3.,   3.,   2.,  41.,  56.]])

In [72]:
Tra_fd = Fd_rg  # 지역별 부문별 최종수요
Tra_fd

array([[  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 29.,  21.,   0.,   9.,   6.,   0.,   7.,   8.,   0.],
       [387., 164.,   0.,  33.,  13.,   0.,  31.,  11.,   0.],
       [  3.,   0.,   0.,   3.,   1.,   0.,   1.,   0.,   0.],
       [ 20.,  15.,   0.,  19.,  11.,   0.,   7.,   7.,   0.],
       [ 11.,   2.,   0., 194.,  77.,   0.,   5.,   1.,   0.],
       [  3.,   0.,   0.,   1.,   0.,   0.,   2.,   1.,   0.],
       [ 15.,   9.,   0.,   7.,   4.,   0.,  15.,   5.,   0.],
       [ 15.,   4.,   0.,   5.,   1.,   0., 146.,  71.,   0.]])

In [73]:
Tot_tra = (Tra_int+Tra_fd).astype(float)  #지역별 산업별 이출입 매트릭스 9x9
Tot_tra

array([[  1.,   3.,   1.,   0.,   0.,   0.,   0.,   1.,   0.],
       [ 31., 133.,  71.,  10.,  42.,  14.,   8.,  45.,  13.],
       [388., 243., 302.,  34.,  40.,  31.,  32.,  39.,  28.],
       [  3.,   1.,   2.,   4.,   8.,   2.,   1.,   2.,   0.],
       [ 20.,  58.,  38.,  23., 169.,  40.,   8.,  43.,  13.],
       [ 11.,   6.,  15., 196., 136.,  76.,   5.,   4.,   4.],
       [  3.,   2.,   3.,   1.,   1.,   1.,   3.,  13.,   2.],
       [ 15.,  55.,  40.,   8.,  45.,  15.,  21.,  93.,  31.],
       [ 15.,   7.,  17.,   5.,   4.,   3., 148., 112.,  56.]])

In [74]:
Cd_for_tr = np.array([[1,1,1,0,0,0,0,0,0],  # 행렬 곱을 이용한 부문 통합을 위해 0과 1로 구성된 임시의 6x6 매트릭스
       [0,0,0,1,1,1,0,0,0],
       [0,0,0,0,0,0,1,1,1]])


In [75]:
Fnl_tra = (Cd_for_tr@Tot_tra)@Cd_for_tr.T # 지역별 매트릭스(자기지역 + 타지역)

In [76]:
Fnl_tra_rw_sm = Fnl_tra.sum(axis=1).reshape([3,1])
Fnl_tra_rw_mtx = np.hstack((Fnl_tra,Fnl_tra_rw_sm))

Fnl_tra_col_sm = Fnl_tra_rw_mtx.sum(axis=0).reshape([1,4])

In [77]:
#지역 간 교역 구조

Fnl_tra_ff = np.vstack((Fnl_tra_rw_mtx,Fnl_tra_col_sm)) # 지역간 이출입 구조
Fnl_tra_ratio = Fnl_tra_ff [:3,:]/(np.tile(Fnl_tra_ff[:3,3], reps=[4,1]).T) # 이출구조
Fnl_inter_ratio = Fnl_tra_ff [:,:3]/(np.tile(Fnl_tra_ff[3,:3], reps=[4,1])) # 이입구조

print('지역 간 이출구조', '\n', (Fnl_tra_ratio*100).round(1))
print('지역 간 이입구조', '\n', (Fnl_inter_ratio*100).round(1))

지역 간 이출구조 
 [[ 77.7  11.3  11.  100. ]
 [ 17.3  73.6   9.  100. ]
 [ 21.8  11.5  66.6 100. ]]
지역 간 이입구조 
 [[ 79.   18.8  22.9]
 [ 10.4  72.   11. ]
 [ 10.6   9.1  66.1]
 [100.  100.  100. ]]


In [78]:
Tra_df = pd.DataFrame(Fnl_tra_ff, columns = ['수도권', '동부권', '서부권', '총이출'],
                                     index=['수도권', '동부권', '서부권', '총이입'])
Tra_df

,수도권,동부권,서부권,총이출
수도권,1173.0,171.0,166.0,1510.0
동부권,154.0,654.0,80.0,888.0
서부권,157.0,83.0,479.0,719.0
총이입,1484.0,908.0,725.0,3117.0


In [79]:
Tra_df.to_csv('지역간 이출입 구조.csv', encoding='utf_8-sig')

###..산업연관효과분석

In [80]:
# 투입계수표

Dm_1 = Mt[0:9,0:9].astype(float)
Im_1 = Mt[9:12,0:9].astype(float)
# Dm_1 = Dt[0:4,1:10].astype(float)
# Im_1 = It[0:4,1:10].astype(float)
# print('총거래표 : \n', Tt_1)    # 총거래표 확인
print('국산거래표 : \n', Dm_1)  # 국산거래표 확인
print('수입거래표 : \n', Im_1.astype(float))  # 수입거래표 확인

국산거래표 : 
 [[  0.   3.   1.   0.   0.   0.   0.   1.   0.]
 [  2. 112.  71.   1.  36.  14.   1.  37.  13.]
 [  1.  79. 302.   1.  27.  31.   1.  28.  28.]
 [  0.   1.   2.   1.   7.   2.   0.   2.   0.]
 [  0.  43.  38.   4. 158.  40.   1.  36.  13.]
 [  0.   4.  15.   2.  59.  76.   0.   3.   4.]
 [  0.   2.   3.   0.   1.   1.   1.  12.   2.]
 [  0.  46.  40.   1.  41.  15.   6.  88.  31.]
 [  0.   3.  17.   0.   3.   3.   2.  41.  56.]]
수입거래표 : 
 [[  0.   3.   0.   0.   2.   0.   0.   3.   0.]
 [  0. 101.  41.   0. 130.  18.   1.  99.  18.]
 [  0.   6.  25.   0.   6.  13.   0.   4.   8.]]


In [81]:
Ct =Mt[-1,:9]  # 총산출액 벡터
Ct_m = np.tile(Ct, reps=[9,1]) # 총산출액을 9번 행반복

In [82]:
Ad =(Dm_1/np.tile(Ct, reps=[9,1]))  # 국산투입계수
Ai =(Im_1/np.tile(Ct, reps=[3,1]))  # 수입투입계수
Va_ratio = Va[:,:9]/Ct  # 부가가치계수

print('국산투입계수 \n',Ad.round(3))
print('수입투입계수 \n',Ai.round(3))
print('부가가치계수 \n', Va_ratio)

국산투입계수 
 [[0.    0.005 0.001 0.    0.    0.    0.    0.002 0.   ]
 [0.333 0.192 0.059 0.043 0.056 0.029 0.034 0.075 0.034]
 [0.167 0.136 0.251 0.043 0.042 0.065 0.034 0.057 0.074]
 [0.    0.002 0.002 0.043 0.011 0.004 0.    0.004 0.   ]
 [0.    0.074 0.032 0.174 0.248 0.084 0.034 0.073 0.034]
 [0.    0.007 0.012 0.087 0.092 0.159 0.    0.006 0.011]
 [0.    0.003 0.002 0.    0.002 0.002 0.034 0.024 0.005]
 [0.    0.079 0.033 0.043 0.064 0.031 0.207 0.179 0.082]
 [0.    0.005 0.014 0.    0.005 0.006 0.069 0.084 0.148]]
수입투입계수 
 [[0.    0.005 0.    0.    0.003 0.    0.    0.006 0.   ]
 [0.    0.174 0.034 0.    0.204 0.038 0.034 0.202 0.047]
 [0.    0.01  0.021 0.    0.009 0.027 0.    0.008 0.021]]
부가가치계수 
 [[0.5   0.308 0.539 0.565 0.263 0.555 0.552 0.279 0.544]]


##### 생산유발계수(Lf)

In [83]:
Lf = np.linalg.inv(np.identity(9)-Ad)    # 생산유발계수
Lf_sm_cl = np.hstack((Lf, Lf.sum(axis=1).reshape([9,1]))) # 생산유발계수의 열합
Lf_sm_rw = np.vstack((Lf_sm_cl,Lf_sm_cl.sum(axis=0))) # 생산유발계수의 행합
print('생산유발계수', '\n', Lf_sm_rw)

생산유발계수 
 [[ 1.003  0.007  0.002  0.001  0.001  0.001  0.001  0.004  0.001  1.02 ]
 [ 0.449  1.289  0.117  0.101  0.127  0.073  0.093  0.151  0.083  2.485]
 [ 0.316  0.262  1.374  0.117  0.129  0.135  0.106  0.152  0.152  2.745]
 [ 0.002  0.005  0.004  1.05   0.018  0.008  0.003  0.008  0.002  1.1  ]
 [ 0.067  0.159  0.083  0.284  1.384  0.157  0.098  0.159  0.087  2.479]
 [ 0.017  0.034  0.032  0.143  0.158  1.21   0.017  0.033  0.029  1.672]
 [ 0.004  0.01   0.006  0.004  0.007  0.005  1.045  0.034  0.011  1.127]
 [ 0.065  0.154  0.08   0.1    0.137  0.075  0.297  1.275  0.144  2.326]
 [ 0.015  0.029  0.032  0.015  0.026  0.02   0.117  0.132  1.192  1.579]
 [ 1.939  1.951  1.731  1.816  1.987  1.685  1.777  1.947  1.701 16.533]]


#### 수입유발계수(Li)

In [84]:
Dig_imp = linalg.block_diag(Mt[9:12,0:3],Mt[9:12,3:6],Mt[9:12,6:9]) # 지역별 수입거래표의 대각행렬

In [85]:
Ai_m = Dig_imp/Ct_m
Ai_m

array([[0.   , 0.005, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.174, 0.034, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.01 , 0.021, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.003, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.204, 0.038, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.009, 0.027, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.006, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.034, 0.202, 0.047],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.008, 0.021]])

In [86]:
Li = Ai_m@Lf  #수입유발계수
Li.shape

Li_sm_cl = np.hstack((Li, Li.sum(axis=1).reshape([9,1]))) # 수입유발계수의 열합
Li_sm_rw = np.vstack((Li_sm_cl,Li_sm_cl.sum(axis=0))) # 수입유발계수의 행합
print('수입유발계수', '\n', Li_sm_rw.round(3))


수입유발계수 
 [[0.002 0.007 0.001 0.001 0.001 0.    0.    0.001 0.    0.013]
 [0.089 0.233 0.067 0.021 0.027 0.017 0.02  0.031 0.02  0.525]
 [0.011 0.019 0.03  0.003 0.004 0.004 0.003 0.005 0.004 0.083]
 [0.    0.    0.    0.001 0.004 0.    0.    0.    0.    0.008]
 [0.014 0.034 0.018 0.063 0.288 0.078 0.021 0.034 0.019 0.568]
 [0.001 0.002 0.002 0.007 0.017 0.034 0.001 0.002 0.002 0.069]
 [0.    0.001 0.    0.001 0.001 0.    0.002 0.008 0.001 0.014]
 [0.014 0.033 0.018 0.021 0.029 0.016 0.101 0.265 0.086 0.583]
 [0.001 0.002 0.001 0.001 0.002 0.001 0.005 0.013 0.026 0.052]
 [0.133 0.33  0.137 0.119 0.372 0.151 0.154 0.359 0.158 1.914]]


#### 부가가치유발계수(Lv)

In [87]:
Va_r_m = Mt[-2,:9]/Ct # 지역별 부문별 부가가치계수

In [88]:
Va_m = np.diag(Va_r_m)  # 지역별 부문별 부가가치율의 대각행렬
Va_m

array([[0.5  , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.308, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.539, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.565, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.263, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.555, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.552, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.279, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.544]])

In [89]:
Lv = Va_m@Lf  #부가가치유발계수
Lv_sm_cl = np.hstack((Lv, Lv.sum(axis=1).reshape([9,1]))) # 부가가치유발계수의 열합
Lv_sm_rw = np.vstack((Lv_sm_cl,Lv_sm_cl.sum(axis=0))) # 부가가치유발계수의 행합
print('부가가치유발계수', '\n', Lv_sm_rw.round(3))

부가가치유발계수 
 [[0.501 0.004 0.001 0.    0.001 0.    0.001 0.002 0.    0.51 ]
 [0.138 0.396 0.036 0.031 0.039 0.023 0.029 0.047 0.026 0.764]
 [0.171 0.141 0.74  0.063 0.07  0.073 0.057 0.082 0.082 1.48 ]
 [0.001 0.003 0.002 0.594 0.01  0.004 0.002 0.004 0.001 0.622]
 [0.018 0.042 0.022 0.075 0.364 0.041 0.026 0.042 0.023 0.653]
 [0.009 0.019 0.018 0.08  0.088 0.672 0.009 0.018 0.016 0.929]
 [0.002 0.005 0.004 0.002 0.004 0.003 0.576 0.019 0.006 0.622]
 [0.018 0.043 0.022 0.028 0.038 0.021 0.083 0.356 0.04  0.649]
 [0.008 0.016 0.018 0.008 0.014 0.011 0.063 0.072 0.648 0.858]
 [0.867 0.67  0.863 0.881 0.628 0.849 0.846 0.641 0.842 7.086]]


In [90]:
# df_Lv = pd.DataFrame(Lv.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  # pandas로 내보내기

In [91]:

Lf_df = pd.DataFrame(Lf.round(3), columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
Lf_df.index.names=['지역', '상품']
Lf_df  #pandas로 내보내기

수도권                  동부권                  서부권              
           농산품    공산품    서비스    농산품    공산품    서비스    농산품    공산품    서비스
지역  상품                                                                
수도권 농산품  1.003  0.007  0.002  0.001  0.001  0.001  0.001  0.004  0.001
    공산품  0.449  1.289  0.117  0.101  0.127  0.073  0.093  0.151  0.083
    서비스  0.316  0.262  1.374  0.117  0.129  0.135  0.106  0.152  0.152
동부권 농산품  0.002  0.005  0.004  1.050  0.018  0.008  0.003  0.008  0.002
    공산품  0.067  0.159  0.083  0.284  1.384  0.157  0.098  0.159  0.087
    서비스  0.017  0.034  0.032  0.143  0.158  1.210  0.017  0.033  0.029
서부권 농산품  0.004  0.010  0.006  0.004  0.007  0.005  1.045  0.034  0.011
    공산품  0.065  0.154  0.080  0.100  0.137  0.075  0.297  1.275  0.144
    서비스  0.015  0.029  0.032  0.015  0.026  0.020  0.117  0.132  1.192

In [92]:
Lv_df = pd.DataFrame(Lv, columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
Lv_df.index.names=['지역', '상품']
# Lv_df  #pandas로 내보내기

In [93]:
Li_df = pd.DataFrame(Li, columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
Li_df.index.names=['지역', '상품']
# Li_df  #pandas로 내보내기

In [94]:
Lf_df.to_csv('생산유발계수.csv', encoding='utf_8-sig')
Lv_df.to_csv('부가가치유발계수.csv', encoding='utf_8-sig')
Li_df.to_csv('수입유발계수.csv', encoding='utf_8-sig')

## 취업계수 및 취업유발계수

In [95]:
Rgt_emp_3

,cd 수도권,동부권,서부권
농산품,135543.0,485642.0,606337.0
공산품,1818636.0,1444100.0,834780.0
서비스,10302991.0,4744657.0,3749541.0


In [96]:
Emp_3 = Rgt_emp_3.to_numpy() # 지역별 부문별 취업자수
Emp = Emp_3[0:4,0:4].astype(float)
Emp

array([[  135543.,   485642.,   606337.],
       [ 1818636.,  1444100.,   834780.],
       [10302991.,  4744657.,  3749541.]])

In [97]:
Emp_row = np.concatenate((Emp[:,0],Emp[:,1],Emp[:,2]))
Emp_row.shape

(9,)

In [98]:
Ct.shape

(9,)

In [99]:
# emp_row = np.vstack(sudo_emp,dongbu_emp)

In [100]:
Emp_ratio = ((Emp_row/Ct)/1000).T # 취업계수(십억원당)
Emp_ratio = Emp_ratio.reshape(1,9)
print('취업계수', '\n', Emp_ratio)

취업계수 
 [[22.59   3.125  8.557 21.115  2.263  9.905 20.908  1.7    9.893]]


In [101]:
Emp_Lf= np.diag(Emp_ratio.flatten())@Lf #취업유발계수
Emp_sm_cl = np.hstack((Emp_Lf, Emp_Lf.sum(axis=1).reshape([9,1]))) # 취업유발계수 열합
Emp_sm_rw = np.vstack((Emp_sm_cl,Emp_sm_cl.sum(axis=0))) # 취업유발계수 행합
print('취업유발계수', '\n', Emp_sm_rw.round(1))

취업유발계수 
 [[ 22.7   0.2   0.    0.    0.    0.    0.    0.1   0.   23. ]
 [  1.4   4.    0.4   0.3   0.4   0.2   0.3   0.5   0.3   7.8]
 [  2.7   2.2  11.8   1.    1.1   1.2   0.9   1.3   1.3  23.5]
 [  0.1   0.1   0.1  22.2   0.4   0.2   0.1   0.2   0.   23.2]
 [  0.2   0.4   0.2   0.6   3.1   0.4   0.2   0.4   0.2   5.6]
 [  0.2   0.3   0.3   1.4   1.6  12.    0.2   0.3   0.3  16.6]
 [  0.1   0.2   0.1   0.1   0.1   0.1  21.8   0.7   0.2  23.6]
 [  0.1   0.3   0.1   0.2   0.2   0.1   0.5   2.2   0.2   4. ]
 [  0.1   0.3   0.3   0.2   0.3   0.2   1.2   1.3  11.8  15.6]
 [ 27.5   8.   13.3  26.    7.2  14.4  25.2   6.9  14.4 142.8]]


In [102]:
Emp_ratio_df = pd.DataFrame(Emp_ratio.round(3), columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['취업계수']])

Emp_Lf_df = pd.DataFrame(Emp_Lf, columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
Emp_Lf_df.index.names=['지역', '상품']

#pandas로 내보내기

In [103]:
Emp_ratio_df

수도권                   동부권                   서부권            
        농산품    공산품    서비스     농산품    공산품    서비스     농산품  공산품    서비스
취업계수  22.59  3.125  8.557  21.115  2.263  9.905  20.908  1.7  9.893

In [104]:
Emp_Lf_df

수도권                             동부권                       \
               농산품       공산품        서비스        농산품       공산품        서비스   
지역  상품                                                                    
수도권 농산품  22.651729  0.162131   0.043109   0.018534  0.023576   0.014534   
    공산품   1.403880  4.028263   0.366757   0.314517  0.398157   0.229486   
    서비스   2.707963  2.246110  11.755560   1.002256  1.107804   1.157607   
동부권 농산품   0.052220  0.114127   0.085067  22.173783  0.371593   0.164150   
    공산품   0.151657  0.360856   0.188230   0.643078  3.131731   0.356425   
    서비스   0.164783  0.336634   0.315429   1.418883  1.566771  11.986428   
서부권 농산품   0.089490  0.201967   0.133007   0.084775  0.146093   0.114745   
    공산품   0.109951  0.262113   0.135482   0.170563  0.233144   0.127269   
    서비스   0.149734  0.288510   0.321382   0.151635  0.254388   0.200098   

               서부권                       
               농산품       공산품        서비스  
지역  상품                                   
수도권 농산품   0.026497  0.079132   0.019150  
    공산품   0.291059  0.472757   0.259963  
    서비스   0.911307  1.301539   1.298939  
동부권 농산품   0.059492  0.167367   0.045388  
    공산품   0.222437  0.358865   0.197119  
    서비스   0.168871  0.323432   0.284477  
서부권 농산품  21.842043  0.716210   0.230943  
    공산품   0.504487  2.167675   0.244420  
    서비스   1.153608  1.308330  11.793385

In [105]:
Emp_ratio_df.to_csv('지역별_상품별_취업계수.csv', encoding='utf_8-sig')
Emp_Lf_df.to_csv('지역별_상품별_취업유발계수.csv', encoding='utf_8-sig')

### Tiva_in and Tiva_out

## 수도권의 Tiva_in and Tiva_out

In [106]:
# 수도권의 타지역에 대한 부가가치 이입 및 이출
Fd = Mt[:9,10:19]
Fd_sudo = Fd[:,0:3].sum(axis=1)
Fd_non_sudo = Fd[:,3:9].sum(axis=1)
Mt

array([[   0.,    3.,    1.,    0.,    0.,    0.,    0.,    1.,    0.,
           5.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    1.,    6.,    0.,    6.],
       [   2.,  112.,   71.,    1.,   36.,   14.,    1.,   37.,   13.,
         287.,   29.,   21.,  215.,    9.,    6.,    0.,    7.,    8.,
           0.,  295.,  582.,    0.,  582.],
       [   1.,   79.,  302.,    1.,   27.,   31.,    1.,   28.,   28.,
         498.,  387.,  164.,   67.,   33.,   13.,    0.,   31.,   11.,
           0.,  706., 1204.,    0., 1204.],
       [   0.,    1.,    2.,    1.,    7.,    2.,    0.,    2.,    0.,
          15.,    3.,    0.,    0.,    3.,    1.,    0.,    1.,    0.,
           0.,    8.,   23.,    0.,   23.],
       [   0.,   43.,   38.,    4.,  158.,   40.,    1.,   36.,   13.,
         333.,   20.,   15.,    0.,   19.,   11.,  226.,    7.,    7.,
           0.,  305.,  638.,    0.,  638.],
       [   0.,    4.,   15.,    2.,   59.,   76.,    0.,    3.,    4.,

In [107]:
Mtx_for_vain = Lv@np.diagflat(Fd_sudo)
Mtx_for_vaout = Lv@np.diagflat(Fd_non_sudo)
Mtx_for_vaout

array([[  0.   ,   0.108,   0.084,   0.002,   0.141,   0.097,   0.002,
          0.349,   0.1  ],
       [  0.   ,  11.895,   3.177,   0.155,  10.581,   6.844,   0.115,
          9.26 ,   6.013],
       [  0.   ,   4.245,  65.164,   0.316,  18.841,  22.095,   0.23 ,
         16.315,  19.228],
       [  0.   ,   0.092,   0.2  ,   2.968,   2.686,   1.331,   0.006,
          0.892,   0.286],
       [  0.   ,   1.259,   1.927,   0.374,  98.369,  12.564,   0.104,
          8.308,   5.389],
       [  0.   ,   0.566,   1.556,   0.398,  23.716, 203.615,   0.038,
          3.608,   3.748],
       [  0.   ,   0.16 ,   0.309,   0.011,   1.041,   0.917,   2.305,
          3.761,   1.432],
       [  0.   ,   1.291,   1.957,   0.14 ,  10.331,   6.329,   0.331,
         70.794,   9.427],
       [  0.   ,   0.476,   1.554,   0.042,   3.774,   3.331,   0.254,
         14.304, 152.263]])

In [108]:
Tiva_in_sudo = Mtx_for_vaout[:3,:] # 타지역 최종수요에 의한 수도권 부가가치유발액
Tiva_in_sudo_sm = Tiva_in_sudo.sum(axis=1).reshape([3,1])
Tiva_in_sudo_all = Tiva_in_sudo_sm.sum()
Tiva_in_sudo_vec = np.vstack((Tiva_in_sudo_sm,Tiva_in_sudo_all))

Tiva_out_sudo = Mtx_for_vain[3:,:] # 수도권 최종수요에 의한 타지역 부가가치유발액
Tiva_out_sudo_sm = Tiva_out_sudo.sum(axis=1).reshape([6,1])
Tiva_out_sudo_arg = Tiva_out_sudo_sm[0,:] + Tiva_out_sudo_sm[3, :]
Tiva_out_sudo_manu = Tiva_out_sudo_sm[1,:] + Tiva_out_sudo_sm[4, :]
Tiva_out_sudo_serv = Tiva_out_sudo_sm[2,:] + Tiva_out_sudo_sm[5, :]
Tiva_out_sudo_all = Tiva_out_sudo_sm.sum()
Tiva_out_sudo_vec = np.vstack((Tiva_out_sudo_arg,Tiva_out_sudo_manu,Tiva_out_sudo_serv,Tiva_out_sudo_all))

In [109]:
Tiva_mt=np.hstack((Tiva_in_sudo_vec, Tiva_out_sudo_vec, (Tiva_in_sudo_vec-Tiva_out_sudo_vec) ))
print('수도권의 타지역에 대한 부가가치 이입 및 이출', '\n' ,Tiva_mt.round(0))

수도권의 타지역에 대한 부가가치 이입 및 이출 
 [[  1.  11. -10.]
 [ 48.  76. -28.]
 [146.  59.  88.]
 [195. 145.  50.]]


In [110]:
Tiva_in_sudo_df = pd.DataFrame(Tiva_in_sudo.round(3), columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['수도권', '수도권', '수도권'],
                                           ['농산품', '공산품', '서비스']])

Tiva_out_sudo_df = pd.DataFrame(Tiva_out_sudo, columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['동부권', '동부권', '동부권','서부권', '서부권', '서부권'],
                                           ['농산품', '공산품', '서비스','농산품', '공산품', '서비스']])

Tiva_in_sudo_df.index.names=['지역', '상품']
Tiva_out_sudo_df.index.names=['지역', '상품']
Tiva_out_sudo_df

수도권                             동부권                       \
              농산품        공산품        서비스       농산품        공산품       서비스   
지역  상품                                                                   
동부권 농산품  0.001398   0.809585   1.407266  1.780689   0.348147  0.057123   
    공산품  0.017643  11.124796  13.532872  0.224438  12.751595  0.539043   
    서비스  0.009238   5.001276  10.928672  0.238641   3.074328  8.735946   
서부권 농산품  0.002361   1.412315   2.169040  0.006711   0.134929  0.039362   
    공산품  0.018045  11.399423  13.741058  0.083976   1.339188  0.271527   
    서비스  0.008226   4.200449  10.911884  0.024993   0.489163  0.142914   

              서부권                       
              농산품       공산품        서비스  
지역  상품                                  
동부권 농산품  0.004778  0.107525   0.023084  
    공산품  0.077632  1.001970   0.435707  
    서비스  0.028402  0.435182   0.303025  
서부권 농산품  1.729101  0.453584   0.115788  
    공산품  0.248382  8.537955   0.762149  
    서비스  0.190138  1.725113  12.310655

In [111]:
Tiva_in_sudo_df.to_csv('부가가치이입.csv', encoding='utf_8-sig')
Tiva_out_sudo_df.to_csv('부가가치이출.csv', encoding='utf_8-sig')